In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import accuracy_score, classification_report
pd.set_option('display.max_columns',None)
import joblib

In [2]:
test_day = pd.read_parquet(f'C:/Users/Samsung/Documents/GitHub-Datas/TCC_Machine_Learning/Datas referências/price_data_D10.parquet')
test_day.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,change_in_price,Smoothed_Close,Prediction,Smoothed_Close_1,Verify,Close_10,Smoothed_Close_10,down_days,up_days,RSI,low_14,high_14,k_percent,r_percent,MACD,MACD_EMA,Price_Rate_Of_Change,On Balance Volume
0,2000-01-05,0.520882,0.520882,0.520882,0.520882,0.279469,985,ABEV3.SA,NaN,0.520882,-1.0,0.520882,-1.0,0.520882,0.499416,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,0
1,2000-01-06,0.494478,0.494478,0.494478,0.494478,0.265302,227,ABEV3.SA,-0.026404,0.515601,-1.0,0.494478,1.0,0.501104,0.499753,0.026404,0.000000,0.000000,NaN,NaN,NaN,NaN,-0.000118,-0.000066,NaN,-227
2,2000-01-12,0.481293,0.481293,0.481293,0.481293,0.258228,1137,ABEV3.SA,-0.013185,0.508740,-1.0,0.481293,1.0,0.514289,0.502661,0.013185,0.000000,0.000000,NaN,NaN,NaN,NaN,-0.000364,-0.000188,NaN,-1364
3,2000-01-13,0.484589,0.484589,0.484589,0.484589,0.259996,606,ABEV3.SA,0.003296,0.503909,-1.0,0.484589,1.0,0.485255,0.499179,0.000000,0.003296,9.538366,NaN,NaN,NaN,NaN,-0.000641,-0.000341,NaN,-1970
4,2000-01-14,0.494478,0.494478,0.494478,0.494478,0.265302,6445,ABEV3.SA,0.009889,0.502023,-1.0,0.494478,-1.0,0.494478,0.498239,0.000000,0.009889,31.994220,NaN,NaN,NaN,NaN,-0.000853,-0.000494,NaN,-8415


In [3]:
test_day['Ticker'].unique()

array(['ABEV3.SA', 'ITUB4.SA', 'MGLU3.SA', 'PETR4.SA', 'VALE3.SA'],
      dtype=object)

In [4]:
test_day_AMBEV = test_day[test_day['Ticker'] == 'ABEV3.SA'].reset_index(drop=True)
test_day_AMBEV['Prediction'] = np.NaN

test_day_AMBEV.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,change_in_price,Smoothed_Close,Prediction,Smoothed_Close_1,Verify,Close_10,Smoothed_Close_10,down_days,up_days,RSI,low_14,high_14,k_percent,r_percent,MACD,MACD_EMA,Price_Rate_Of_Change,On Balance Volume
0,2000-01-05,0.520882,0.520882,0.520882,0.520882,0.279469,985,ABEV3.SA,NaN,0.520882,NaN,0.520882,-1.0,0.520882,0.499416,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,0
1,2000-01-06,0.494478,0.494478,0.494478,0.494478,0.265302,227,ABEV3.SA,-0.026404,0.515601,NaN,0.494478,1.0,0.501104,0.499753,0.026404,0.000000,0.000000,NaN,NaN,NaN,NaN,-0.000118,-0.000066,NaN,-227
2,2000-01-12,0.481293,0.481293,0.481293,0.481293,0.258228,1137,ABEV3.SA,-0.013185,0.508740,NaN,0.481293,1.0,0.514289,0.502661,0.013185,0.000000,0.000000,NaN,NaN,NaN,NaN,-0.000364,-0.000188,NaN,-1364
3,2000-01-13,0.484589,0.484589,0.484589,0.484589,0.259996,606,ABEV3.SA,0.003296,0.503909,NaN,0.484589,1.0,0.485255,0.499179,0.000000,0.003296,9.538366,NaN,NaN,NaN,NaN,-0.000641,-0.000341,NaN,-1970
4,2000-01-14,0.494478,0.494478,0.494478,0.494478,0.265302,6445,ABEV3.SA,0.009889,0.502023,NaN,0.494478,-1.0,0.494478,0.498239,0.000000,0.009889,31.994220,NaN,NaN,NaN,NaN,-0.000853,-0.000494,NaN,-8415


In [5]:
test_day_AMBEV['Date'] = pd.to_datetime(test_day_AMBEV['Date'])

# Definir a data inicial para filtrar
inicio = '2024-12-05'  # Data inicial

# Filtrar as linhas onde a data é maior ou igual a 'inicio'
test_day_AMBEV = test_day_AMBEV[test_day_AMBEV['Date'] >= inicio]

test_day_AMBEV.head(15)


,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,change_in_price,Smoothed_Close,Prediction,Smoothed_Close_1,Verify,Close_10,Smoothed_Close_10,down_days,up_days,RSI,low_14,high_14,k_percent,r_percent,MACD,MACD_EMA,Price_Rate_Of_Change,On Balance Volume
5717,2024-12-05,14.05,14.46,14.00,14.42,13.666730,38794000,ABEV3.SA,0.490000,13.389995,NaN,14.42,-1.0,12.78,13.137559,0.000000,0.49,86.722945,12.25,14.46,51.583477,-48.416523,0.052492,-0.040186,0.066920,-2122382047
5718,2024-12-06,14.30,14.53,14.15,14.22,13.477178,46449200,ABEV3.SA,-0.200000,13.555996,NaN,14.22,-1.0,12.23,12.956047,0.200000,0.00,77.627058,12.25,14.53,57.280530,-42.719470,0.106557,-0.010837,0.080322,-2075932847
5719,2024-12-09,14.22,14.33,14.14,14.21,13.467700,25321800,ABEV3.SA,-0.010000,13.686797,NaN,14.21,-1.0,12.22,12.808838,0.010000,0.00,77.160149,12.25,14.53,63.017409,-36.982591,0.158135,0.022957,0.089671,-2050611047
5720,2024-12-10,14.33,14.50,14.27,14.27,13.524566,22942500,ABEV3.SA,0.060000,13.803438,NaN,14.27,-1.0,12.14,12.675070,0.000000,0.06,78.073184,12.25,14.53,68.133232,-31.866768,0.206047,0.059575,0.099141,-2027668547
5721,2024-12-11,14.27,14.36,13.66,13.80,13.079118,49150600,ABEV3.SA,-0.470000,13.802750,NaN,13.80,-1.0,12.07,12.554056,0.470000,0.00,57.351316,12.25,14.53,68.103080,-31.896920,0.241183,0.095897,0.101865,-2076819147
5722,2024-12-12,13.50,13.58,13.10,13.10,12.415685,49616900,ABEV3.SA,-0.700000,13.662200,NaN,13.10,-1.0,11.88,12.419245,0.700000,0.00,39.386518,12.25,14.53,61.938608,-38.061392,0.254750,0.127667,0.087117,-2126436047
5723,2024-12-13,13.10,13.36,13.01,13.08,12.396729,30867900,ABEV3.SA,-0.020000,13.545760,NaN,13.08,NaN,NaN,NaN,0.020000,0.00,38.983936,12.25,14.53,56.831589,-43.168411,0.253188,0.152772,0.066268,-2157303947
5724,2024-12-16,13.08,13.42,12.84,12.84,12.169266,32095500,ABEV3.SA,-0.240000,13.404608,NaN,12.84,NaN,NaN,NaN,0.240000,0.00,34.150823,12.25,14.53,50.640711,-49.359289,0.237819,0.169781,0.036456,-2189399447
5725,2024-12-17,13.00,13.19,12.99,13.10,12.415685,39564500,ABEV3.SA,0.260000,13.343687,NaN,13.10,NaN,NaN,NaN,0.000000,0.26,42.986323,12.25,14.53,47.968714,-52.031286,0.218208,0.179466,0.016082,-2228963947
5726,2024-12-18,13.07,13.08,12.75,12.76,12.093445,40864200,ABEV3.SA,-0.340000,13.226949,NaN,12.76,NaN,NaN,NaN,0.340000,0.00,35.748723,12.25,14.53,42.848658,-57.151342,0.191043,0.181782,-0.012177,-2269828147


##### Carregar modelo aprendido geral

In [6]:
# Carregar o modelo salvo
rand_frst_clf_Ambev = joblib.load('C:/Users/Samsung/Documents/GitHub-Datas/TCC_Machine_Learning/random_forest_model.pkl')

#### Carregar modelo aprendido especifico.

In [17]:
# Carregar o modelo salvo
rand_frst_clf_Ambev = joblib.load('C:/Users/Samsung/Documents/GitHub-Datas/TCC_Machine_Learning/random_forest_model_Ambev.pkl')

In [7]:
Features = test_day_AMBEV[['RSI','k_percent','r_percent','Price_Rate_Of_Change','MACD','MACD_EMA','On Balance Volume']]
Features.head()

,RSI,k_percent,r_percent,Price_Rate_Of_Change,MACD,MACD_EMA,On Balance Volume
5717,86.722945,51.583477,-48.416523,0.066920,0.052492,-0.040186,-2122382047
5718,77.627058,57.280530,-42.719470,0.080322,0.106557,-0.010837,-2075932847
5719,77.160149,63.017409,-36.982591,0.089671,0.158135,0.022957,-2050611047
5720,78.073184,68.133232,-31.866768,0.099141,0.206047,0.059575,-2027668547
5721,57.351316,68.103080,-31.896920,0.101865,0.241183,0.095897,-2076819147


In [8]:
Features_array = Features.to_numpy()

# Fazer previsões usando o modelo carregado
predictions = rand_frst_clf_Ambev.predict(Features_array)

# Adicionar as previsões em uma nova coluna chamada 'Prediction'
test_day_AMBEV['Prediction'] = predictions

test_day_AMBEV.tail(30)

c:\ProgramData\anaconda3\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,change_in_price,Smoothed_Close,Prediction,Smoothed_Close_1,Verify,Close_10,Smoothed_Close_10,down_days,up_days,RSI,low_14,high_14,k_percent,r_percent,MACD,MACD_EMA,Price_Rate_Of_Change,On Balance Volume
5717,2024-12-05,14.05,14.46,14.00,14.42,13.666730,38794000,ABEV3.SA,0.490000,13.389995,1.0,14.42,-1.0,12.78,13.137559,0.000000,0.49,86.722945,12.25,14.46,51.583477,-48.416523,0.052492,-0.040186,0.066920,-2122382047
5718,2024-12-06,14.30,14.53,14.15,14.22,13.477178,46449200,ABEV3.SA,-0.200000,13.555996,-1.0,14.22,-1.0,12.23,12.956047,0.200000,0.00,77.627058,12.25,14.53,57.280530,-42.719470,0.106557,-0.010837,0.080322,-2075932847
5719,2024-12-09,14.22,14.33,14.14,14.21,13.467700,25321800,ABEV3.SA,-0.010000,13.686797,-1.0,14.21,-1.0,12.22,12.808838,0.010000,0.00,77.160149,12.25,14.53,63.017409,-36.982591,0.158135,0.022957,0.089671,-2050611047
5720,2024-12-10,14.33,14.50,14.27,14.27,13.524566,22942500,ABEV3.SA,0.060000,13.803438,-1.0,14.27,-1.0,12.14,12.675070,0.000000,0.06,78.073184,12.25,14.53,68.133232,-31.866768,0.206047,0.059575,0.099141,-2027668547
5721,2024-12-11,14.27,14.36,13.66,13.80,13.079118,49150600,ABEV3.SA,-0.470000,13.802750,-1.0,13.80,-1.0,12.07,12.554056,0.470000,0.00,57.351316,12.25,14.53,68.103080,-31.896920,0.241183,0.095897,0.101865,-2076819147
5722,2024-12-12,13.50,13.58,13.10,13.10,12.415685,49616900,ABEV3.SA,-0.700000,13.662200,-1.0,13.10,-1.0,11.88,12.419245,0.700000,0.00,39.386518,12.25,14.53,61.938608,-38.061392,0.254750,0.127667,0.087117,-2126436047
5723,2024-12-13,13.10,13.36,13.01,13.08,12.396729,30867900,ABEV3.SA,-0.020000,13.545760,-1.0,13.08,NaN,NaN,NaN,0.020000,0.00,38.983936,12.25,14.53,56.831589,-43.168411,0.253188,0.152772,0.066268,-2157303947
5724,2024-12-16,13.08,13.42,12.84,12.84,12.169266,32095500,ABEV3.SA,-0.240000,13.404608,-1.0,12.84,NaN,NaN,NaN,0.240000,0.00,34.150823,12.25,14.53,50.640711,-49.359289,0.237819,0.169781,0.036456,-2189399447
5725,2024-12-17,13.00,13.19,12.99,13.10,12.415685,39564500,ABEV3.SA,0.260000,13.343687,-1.0,13.10,NaN,NaN,NaN,0.000000,0.26,42.986323,12.25,14.53,47.968714,-52.031286,0.218208,0.179466,0.016082,-2228963947
5726,2024-12-18,13.07,13.08,12.75,12.76,12.093445,40864200,ABEV3.SA,-0.340000,13.226949,-1.0,12.76,NaN,NaN,NaN,0.340000,0.00,35.748723,12.25,14.53,42.848658,-57.151342,0.191043,0.181782,-0.012177,-2269828147


In [9]:
test_day_AMBEV.to_excel('C:/Users/Samsung/Documents/GitHub-Datas/TCC_Machine_Learning/Datas referências/test_day.xlsx', index=False)

In [10]:
# We need to remove all rows that have an NaN value.
print('Antes de tirar as NaN nós tinhamos {} linhas e {} colunas'.format(test_day_AMBEV.shape[0], test_day_AMBEV.shape[1]))

# Any row that has a `NaN` value will be dropped.
test_day_AMBEV = test_day_AMBEV.dropna()

# Display how much we have left now.
print('Depois da retirada dos NaN temos {} linhas e {} colunas'.format(test_day_AMBEV.shape[0], test_day_AMBEV.shape[1]))


Antes de tirar as NaN nós tinhamos 16 linhas e 26 colunas
Depois da retirada dos NaN temos 6 linhas e 26 colunas


In [11]:
# Comparar valores e calcular a precisão
test_day_AMBEV['Match'] = (test_day_AMBEV['Prediction'] == test_day_AMBEV['Verify']).astype(int)
accuracy = test_day_AMBEV['Match'].mean()  # Proporção de acertos (iguais / total)

print(f"\nPrecisão do modelo: {accuracy:.2%}")


Precisão do modelo: 83.33%
